In [1]:
# Model Fusion with concatination RGB, GPS, and LiDAR Transformers: DeepSense Scenario 36 64 Beams!
# MLP

# Average Top-1 accuracy 0.14637096774193548
# Average Top-3 accuracy 0.23911290322580644
# Average Top-5 accuracy 0.29274193548387095
# Average Top-7 accuracy 0.34596774193548385
# Average Top-9 accuracy 0.39838709677419354
# Average Top-11 accuracy 0.43709677419354837
# Average Top-13 accuracy 0.4717741935483871
# Average Top-15 accuracy 0.4963709677419355

In [2]:
# !pip install timm

In [3]:
# !pip install plyfile

In [4]:
import torch
import torch.nn as nn
import torch.optim as optimizer
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms as transf
import torch.nn.functional as F
import timm
import pandas as pd
import numpy as np
import ast
from skimage import io
import os
import datetime
import torch as t
import pandas as pd
import torch.cuda as cuda
from torchsummary import summary
from torch.utils.model_zoo import load_url as load_state_dict_from_url
import matplotlib.pyplot as plt
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from plyfile import PlyData, PlyElement
from sklearn.metrics import f1_score


In [5]:
# Training Hyper-parameters
batch_size = 4 # 16, 32 out of memory problem!
val_batch_size = 1
lr = 0.001
decay = 1e-4
num_epochs = 1
train_size = [1]

# Hyperparameters for our network
input_size = 2
node = 512
output_size = 65

In [6]:
# year month day
dayTime = datetime.datetime.now().strftime('%m-%d-%Y')
# Minutes and seconds
hourTime = datetime.datetime.now().strftime('%H_%M')
print(dayTime + '\n' + hourTime)

03-19-2025
08_46


In [7]:
# !cp -r /kaggle/input/privatedata/* /kaggle/working/

In [8]:
# %cd /kaggle/working/

In [9]:
pwd = os.getcwd() + '//' + 'saved_folder' + '//' + dayTime + '_' + hourTime
print(pwd)
# Determine whether the folder already exists
isExists = os.path.exists(pwd)
if not isExists:
    os.makedirs(pwd)

C:\Users\Baqer\Desktop\V2X_CNN_All\Scenario36_64-Beams\Main_Folder//saved_folder//03-19-2025_08_46


In [10]:
save_directory = pwd + '//' + 'saved_analysis_files'
checkpoint_directory = pwd + '//' + 'checkpoint'

isExists = os.path.exists(save_directory)
if not isExists:
    os.makedirs(save_directory)

isExists = os.path.exists(checkpoint_directory)
if not isExists:
    os.makedirs(checkpoint_directory)

In [11]:
# !pip install GPUtil

import torch
from GPUtil import showUtilization as gpu_usage
from numba import cuda

def free_gpu_cache():
    print("Initial GPU Usage")
    gpu_usage()                             

    torch.cuda.empty_cache()

    cuda.select_device(0)
    cuda.close()
    cuda.select_device(0)

    print("GPU Usage after emptying the cache")
    gpu_usage()

free_gpu_cache()

Initial GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  1% | 26% |
GPU Usage after emptying the cache
| ID | GPU | MEM |
------------------
|  0 |  1% | 27% |


In [12]:
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [13]:
import torch

print("CUDA Available:", torch.cuda.is_available())
print("CUDA Device Count:", torch.cuda.device_count())
print("CUDA Device Name:", torch.cuda.get_device_name(0))

CUDA Available: True
CUDA Device Count: 1
CUDA Device Name: NVIDIA GeForce RTX 2080 Super with Max-Q Design


In [14]:
torch.cuda.memory_allocated()

0

In [15]:
torch.cuda.max_memory_allocated()

0

In [16]:
# import os 
# os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:64"

In [17]:
def create_samples(root, shuffle=False, nat_sort=False):
    f = pd.read_csv(root)
    data_samples = []
    pred_val = []
    for idx, row in f.iterrows():
        data = list(row.values[1:])
        # print(data)
        data_samples.append(data)

    return data_samples


class DataFeed(Dataset):
    '''
    A class retrieving a tuple of (image,label). It can handle the case
    of empty classes (empty folders).
    '''
    def __init__(self, image_dir, pos_dir, lidar_dir, nat_sort = False, num_points=15000, image_transform=None, pos_transform = None, init_shuflle = True):
        self.root1 = image_dir
        self.root2 = pos_dir
        self.root3 = lidar_dir

        self.num_points = num_points

        self.samples1 = create_samples(self.root1, shuffle=init_shuflle, nat_sort=nat_sort)
        self.samples2 = create_samples(self.root2, shuffle=init_shuflle, nat_sort=nat_sort)
        self.samples3 = create_samples(self.root3, shuffle=init_shuflle, nat_sort=nat_sort)
        self.transform1 = image_transform
        self.transform2 = pos_transform


    def __len__(self):
        return len(self.samples1)

    def __getitem__(self, idx):
        sample1 = self.samples1[idx]
        img = io.imread(sample1[0])
        if self.transform1:
            img = self.transform1(img)
        label = sample1[1]

        ################################
        sample2 = self.samples2[idx]

        pos_val = sample2[:1]
        pos_val = ast.literal_eval(pos_val[0])
        pos_val = np.asarray(pos_val)
        ################################
        sample3 = self.samples3[idx]
        df = pd.read_csv(sample3[0])
        # Extract the columns of interest
        x = df[' X (mm)'] / 1000  # Convert to meters
        y = df[' Y (mm)'] / 1000
        z = df[' Z (mm)'] / 1000
        points = np.column_stack((x.values, y.values, z.values))
        if points.shape[0] < self.num_points:
            points = np.pad(points, ((0, self.num_points - points.shape[0]), (0, 0)), mode='constant')
        elif points.shape[0] > self.num_points:
            indices = np.random.choice(points.shape[0], self.num_points, replace=False)
            points = points[indices]

        points = torch.tensor(points, dtype=torch.float32)

        # pos_centers = sample2[1:2]
        # pos_centers = np.asarray(pos_centers)

        return ([img, pos_val, points.T], label)

In [18]:
# Image transform
img_resize = transf.Resize((224, 224))
img_norm = transf.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
image_proc_pipe = transf.Compose(
    [transf.ToPILImage(),
     img_resize,
     transf.ToTensor(),
     img_norm]
)


#POS transform
pos_proc_pipe = transf.Compose(
    [
        transf.ToTensor()
    ]
)

In [19]:
image_train_dir = './scenario36_64_img_beam_train.csv'
image_val_dir = './scenario36_64_img_beam_val.csv'

pos_train_dir = './scenario36_64_pos_beam_train.csv'
pos_val_dir = './scenario36_64_pos_beam_val.csv'

lidar_train_dir = './scenario36_64_lidar_beam_train.csv'
lidar_val_dir = './scenario36_64_lidar_beam_val.csv'

train_loader = DataLoader(DataFeed(image_train_dir, pos_train_dir, lidar_train_dir, image_transform=image_proc_pipe),
                          batch_size=batch_size,
                          #num_workers=8,
                          shuffle=False)
val_loader = DataLoader(DataFeed(image_val_dir, pos_val_dir, lidar_train_dir, image_transform=image_proc_pipe),
                        batch_size=val_batch_size,
                        #num_workers=8,
                        shuffle=False)

In [20]:
                        ####PointTransformerV3####


# import torch
# import torch.nn as nn
import torch.nn.functional as F
# from torchsummary import summary

class InputLayer(nn.Module):
    """Input Layer to accept input point cloud data."""
    def __init__(self, k=3):
        super(InputLayer, self).__init__()
        self.k = k

    def forward(self, x):
        # Ensure the input shape is (batch_size, num_points, channels)
        return x.transpose(1, 2)  # Change shape to (batch_size, channels, num_points)


class EmbeddingLayer(nn.Module):
    """Embedding Layer for dimensionality reduction."""
    def __init__(self, input_channels, output_channels):
        super(EmbeddingLayer, self).__init__()
        self.conv = nn.Conv1d(input_channels, output_channels, kernel_size=1)

    def forward(self, x):
        # x should be in shape (batch_size, channels, num_points)
        x = x.transpose(1, 2)  # Transpose to (batch_size, channels, num_points)
        return F.relu(self.conv(x))  # Applies a 1x1 convolution


class EncoderStage(nn.Module):
    def __init__(self, in_channels, out_channels, dropout_rate=0.3):
        super(EncoderStage, self).__init__()
        self.cpe = nn.Conv1d(in_channels, out_channels, kernel_size=1)  # CPE
        self.norm = nn.BatchNorm1d(out_channels)  # Normalization
        self.attention = nn.MultiheadAttention(embed_dim=out_channels, num_heads=8)  # Attention
        self.dropout_att = nn.Dropout(dropout_rate)  # Dropout after attention
        self.mlp = nn.Sequential(
            nn.Linear(out_channels, out_channels),  # MLP layer
            nn.ReLU(),
            nn.Dropout(dropout_rate),  # Dropout after first MLP layer
            nn.Linear(out_channels, out_channels)   # Another MLP layer
        )
        self.dropout_mlp = nn.Dropout(dropout_rate)  # Dropout after second MLP layer

    def forward(self, x):
        # Apply CPE
        identity = x  # Save the input for residual connection
        x = F.relu(self.cpe(x))
        x = self.norm(x)

        # Prepare for Attention mechanism
        x_att = x.transpose(2, 1)  # Shape change for attention
        x_att, _ = self.attention(x_att, x_att, x_att)  # Attention output
        x_att = self.dropout_att(x_att)  # Apply dropout

        x_att = x_att.transpose(2, 1)  # Transpose back
        x = x + x_att  # Skip connection

        # MLP
        b, c, p = x.size()
        x = x.view(b * p, c)  # Flatten for MLP
        x = self.mlp(x)  # Apply MLP
        x = self.dropout_mlp(x)  # Apply dropout after MLP

        # Reshape back to (B, out_channels, num_points)
        output_channels = x.size(1)
        x = x.view(b, output_channels, p)
        return x


class SerializedPoolingLayer(nn.Module):
    """Serialized Pooling Layer for dimensionality reduction."""
    def __init__(self):
        super(SerializedPoolingLayer, self).__init__()

    def forward(self, x):
        return F.avg_pool1d(x, kernel_size=2)  # Average pooling


class PointTransformerV3(nn.Module):
    """Main Point Transformer V3 for Classification."""
    def __init__(self, num_classes,  enc_depths=[64, 128,256,512], dropout_rate=0.1):
        super(PointTransformerV3, self).__init__()

        # Input and embedding
        self.input_layer = InputLayer()
        self.embedding = EmbeddingLayer(input_channels=3, output_channels=enc_depths[0])  # Initial embedding layer

        # Encoder stages
        self.encoders = nn.ModuleList()
        in_channels = enc_depths[0]
        for out_channels in enc_depths:
            self.encoders.append(EncoderStage(in_channels, out_channels, dropout_rate=dropout_rate))
            in_channels = out_channels

        # Serialized pooling
        self.serialized_pooling = SerializedPoolingLayer()

        # Fully connected layers
        self.fc1 = nn.Linear(enc_depths[-1], 128)  # Input size from enc_depths[-1]
        self.fc2 = nn.Linear(128, 64)  # Intermediate layer
        self.fc3 = nn.Linear(64, 128)  # Output layer for 65 classes

    def forward(self, x):
        # Input layer
        x = self.input_layer(x)

        # Embedding layer
        x = self.embedding(x)

        # Pass through encoder stages
        for encoder in self.encoders:
            x = encoder(x)

        # Serialized pooling
        x = self.serialized_pooling(x)

        # Global feature extraction
        x = torch.max(x, dim=2)[0]  # Global max pooling across points

        x = x.view(x.size(0), -1)  # Flatten the output

        # Fully connected layers
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)

        return x


class FusionModel(nn.Module):
    def __init__(self, num_features, num_classes):
        super(FusionModel, self).__init__()
        self.nclass = num_classes
        self.nf = num_features

        # Using MaxViT from timm for image feature extraction
        self.maxvit = timm.create_model('maxvit_tiny_tf_224.in1k', pretrained=True)
        self.maxvit_fc = nn.Linear(self.maxvit.num_features, 128)

        # Using PointTransformerV3 for point cloud feature extraction
        self.point_transformer = PointTransformerV3(num_classes=self.nclass)

        # MLP for the positional data (additional features)
        self.positional_fc = nn.Sequential(
            nn.Linear(self.nf, 128),
            nn.ReLU(),
            nn.Linear(128, 128)
            
        )

        # Final fully connected layers for fusion
        self.fc1 = nn.Linear(128 + 128 + 128, 256)  # Image + PointCloud + Positional
        self.fc2 = nn.Linear(256, self.nclass)

    def forward(self, inputs):
       
        # Extract image data and pass it through MaxViT
        image_data = inputs[0]
        x_image = self.maxvit.forward_features(image_data)
        x_image = x_image.mean(dim=[2, 3])  # Global average pooling
        x_image = self.maxvit_fc(x_image)

        # Extract point cloud data and pass it through PointTransformerV3
        point_cloud_data = inputs[2]
        x_point_cloud = self.point_transformer(point_cloud_data)

        # Process positional data (additional features)
        positional_data = inputs[1]
        y = self.positional_fc(positional_data.reshape([-1, self.nf]))

        # Concatenate features from image, point cloud, and positional data
        x = torch.cat([x_image, x_point_cloud, y], dim=1)

        # Pass through fully connected layers
        x = self.fc1(x)
        x = self.fc2(x)

        return x

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = FusionModel(num_features=input_size, num_classes=output_size).to(device)

In [21]:
    val_acc = []

#with cuda.device(0):
    top_1 = np.zeros( (1,len(train_size)) )
    top_3 = np.zeros( (1,len(train_size)) )
    top_5 = np.zeros( (1,len(train_size)) )
    top_7 = np.zeros( (1,len(train_size)) )
    top_9 = np.zeros( (1,len(train_size)) )
    top_11 = np.zeros( (1,len(train_size)) )
    top_13 = np.zeros( (1,len(train_size)) )
    top_15 = np.zeros( (1,len(train_size)) )
    acc_loss = 0
    itr = []
    for idx, n in enumerate(train_size):
        print('```````````````````````````````````````````````````````')
        print('Training size is {}'.format(n))
        # Build the network:
        net = model.cuda()
        layers = list(net.children())

        #  Optimization parameters:
        criterion = nn.CrossEntropyLoss()
        opt = optimizer.Adam(net.parameters(), lr=lr, weight_decay=decay)
        LR_sch = optimizer.lr_scheduler.MultiStepLR(opt, [15,25,40], gamma=0.1, last_epoch=-1)

        count = 0
        running_loss = []
        running_top1_acc = []
        running_top3_acc = []
        running_top5_acc = []
        running_top7_acc = []
        running_top9_acc = []
        running_top11_acc = []
        running_top13_acc = []
        running_top15_acc = []

        best_accuracy = 0

        for epoch in range(num_epochs):
            print('Epoch No. ' + str(epoch + 1))
            skipped_batches = 0
            epoch_train_loss = 0  # To track the training loss for the epoch # Added
            for tr_count, (pos_data, beam_val) in enumerate(train_loader):
                net.train()
                data = [pos_data[0], pos_data[1].type(torch.Tensor), pos_data[2].type(torch.Tensor)]
                label = beam_val.type(torch.LongTensor)
                x = [data[0].cuda(), data[1].cuda(), data[2].cuda()]
                #print("x size", x.size())
                opt.zero_grad()
                label = label.cuda()
                #print("label size", label.size())
                # print(x.shape)
                out = net.forward(x)
                # print(out.shape, label.shape)
                loss = criterion(out, label)
                loss.backward()
                opt.step()
                batch_loss = loss.item()
                acc_loss += batch_loss
                epoch_train_loss += batch_loss  # Accumulate batch loss for the epoch # Added
                count += 1
                if np.mod(count, 100) == 0:
                    print('Training-Batch No.' + str(count))
                    running_loss.append(batch_loss)  # running_loss.append()
                    itr.append(count)
                    print('Loss = ' + str(running_loss[-1]))

            epoch_train_loss /= len(train_loader)  # Calculate average training loss for the epoch # Added
            print(f'Epoch {epoch + 1} Training Loss: {epoch_train_loss:.4f}') # Added
            
            print('Start Training and Validation')
            ave_top1_acc = 0
            ave_top3_acc = 0
            ave_top5_acc = 0
            ave_top7_acc = 0
            ave_top9_acc = 0
            ave_top11_acc = 0
            ave_top13_acc = 0
            ave_top15_acc = 0
            ind_ten = t.as_tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14], device='cuda:0')
            top1_pred_out = []
            top3_pred_out = []
            top5_pred_out = []
            top7_pred_out = []
            top9_pred_out = []
            top11_pred_out = []
            top13_pred_out = []
            top15_pred_out = []
            total_count = 0

            gt_beam = []
            for val_count, (pos_data, beam_val) in enumerate(val_loader):
                net.eval()
                data = [pos_data[0], pos_data[1].type(torch.Tensor), pos_data[2].type(torch.Tensor)]
                label = beam_val.type(torch.LongTensor)
                x = [data[0].cuda(), data[1].cuda(), data[2].cuda()]
                #print("x size", x.size())
                opt.zero_grad()
                labels = label.cuda()
                
                # val_batch_loss = criterion(out, labels).item()  # Calculate validation loss for the batch # Added
                # val_loss += val_batch_loss  # Accumulate batch loss for the epoch # Added
                
                gt_beam.append(labels.detach().cpu().numpy()[0].tolist())
                total_count += labels.size(0)
                out = net.forward(x)
                _, top_1_pred = t.max(out, dim=1)
                top1_pred_out.append(top_1_pred.detach().cpu().numpy()[0].tolist())
                sorted_out = t.argsort(out, dim=1, descending=True)

                top_3_pred = t.index_select(sorted_out, dim=1, index=ind_ten[0:3])
                top3_pred_out.append(top_3_pred.detach().cpu().numpy()[0].tolist())

                top_5_pred = t.index_select(sorted_out, dim=1, index=ind_ten[0:5])
                top5_pred_out.append(top_5_pred.detach().cpu().numpy()[0].tolist())

                top_7_pred = t.index_select(sorted_out, dim=1, index=ind_ten[0:7])
                top7_pred_out.append(top_7_pred.detach().cpu().numpy()[0].tolist())

                top_9_pred = t.index_select(sorted_out, dim=1, index=ind_ten[0:9])
                top9_pred_out.append(top_9_pred.detach().cpu().numpy()[0].tolist())

                top_11_pred = t.index_select(sorted_out, dim=1, index=ind_ten[0:11])
                top11_pred_out.append(top_11_pred.detach().cpu().numpy()[0].tolist())

                top_13_pred = t.index_select(sorted_out, dim=1, index=ind_ten[0:13])
                top13_pred_out.append(top_13_pred.detach().cpu().numpy()[0].tolist())

                top_15_pred = t.index_select(sorted_out, dim=1, index=ind_ten[0:15])
                top15_pred_out.append(top_15_pred.detach().cpu().numpy()[0].tolist())

                reshaped_labels = labels.reshape((labels.shape[0], 1))
                tiled_3_labels = reshaped_labels.repeat(1, 3)
                tiled_5_labels = reshaped_labels.repeat(1, 5)
                tiled_7_labels = reshaped_labels.repeat(1, 7)
                tiled_9_labels = reshaped_labels.repeat(1, 9)
                tiled_11_labels = reshaped_labels.repeat(1, 11)
                tiled_13_labels = reshaped_labels.repeat(1, 13)
                tiled_15_labels = reshaped_labels.repeat(1, 15)

                batch_top1_acc = t.sum(top_1_pred == labels, dtype=t.float32)
                batch_top3_acc = t.sum(top_3_pred == tiled_3_labels, dtype=t.float32)
                batch_top5_acc = t.sum(top_5_pred == tiled_5_labels, dtype=t.float32)
                batch_top7_acc = t.sum(top_7_pred == tiled_7_labels, dtype=t.float32)
                batch_top9_acc = t.sum(top_9_pred == tiled_9_labels, dtype=t.float32)
                batch_top11_acc = t.sum(top_11_pred == tiled_11_labels, dtype=t.float32)
                batch_top13_acc = t.sum(top_13_pred == tiled_13_labels, dtype=t.float32)
                batch_top15_acc = t.sum(top_15_pred == tiled_15_labels, dtype=t.float32)

                ave_top1_acc += batch_top1_acc.item()
                ave_top3_acc += batch_top3_acc.item()
                ave_top5_acc += batch_top5_acc.item()
                ave_top7_acc += batch_top7_acc.item()
                ave_top9_acc += batch_top9_acc.item()
                ave_top11_acc += batch_top11_acc.item()
                ave_top13_acc += batch_top13_acc.item()
                ave_top15_acc += batch_top15_acc.item()

            # val_loss /= len(val_loader)  # Calculate average validation loss for the epoch # Added
            # print(f'Epoch {epoch + 1} Validation Loss: {val_loss:.4f}') # Added
            
            print("total training examples are", total_count)
            running_top1_acc.append(ave_top1_acc / total_count)  # (batch_size * (count_2 + 1)) )
            running_top3_acc.append(ave_top3_acc / total_count)
            running_top5_acc.append(ave_top5_acc / total_count)
            running_top7_acc.append(ave_top7_acc / total_count)
            running_top9_acc.append(ave_top9_acc / total_count)
            running_top11_acc.append(ave_top11_acc / total_count)
            running_top13_acc.append(ave_top13_acc / total_count)
            running_top15_acc.append(ave_top15_acc / total_count)

            print('Training_size {}--No. of skipped batchess {}'.format(n,skipped_batches))
            print('Average Top-1 accuracy {}'.format( running_top1_acc[-1]))
            print('Average Top-3 accuracy {}'.format( running_top3_acc[-1]))
            print('Average Top-5 accuracy {}'.format( running_top5_acc[-1]))
            print('Average Top-7 accuracy {}'.format( running_top7_acc[-1]))
            print('Average Top-9 accuracy {}'.format( running_top9_acc[-1]))
            print('Average Top-11 accuracy {}'.format( running_top11_acc[-1]))
            print('Average Top-13 accuracy {}'.format( running_top13_acc[-1]))
            print('Average Top-15 accuracy {}'.format( running_top15_acc[-1]))

            cur_accuracy  = running_top1_acc[-1]

            print("current acc", cur_accuracy)
            print("best acc", best_accuracy)
            if cur_accuracy > best_accuracy:
                print("Saving the best model")
                net_name = checkpoint_directory  + '//' +  'fused_transfomer_beam_pred'
                t.save(net.state_dict(), net_name)
                best_accuracy =  cur_accuracy
            print("updated best acc", best_accuracy)


            print("Saving the predicted value in a csv file")
            file_to_save = f'{save_directory}//topk_fused_val_after_{epoch+1}th_epoch.csv'
            indx = np.arange(1, len(top1_pred_out)+1, 1)
            df1 = pd.DataFrame()
            df1['index'] = indx
            df1['link_status'] = gt_beam
            df1['top1_pred'] = top1_pred_out
            df1['top3_pred'] = top3_pred_out
            df1['top5_pred'] = top5_pred_out
            df1['top7_pred'] = top7_pred_out
            df1['top9_pred'] = top9_pred_out
            df1['top11_pred'] = top11_pred_out
            df1['top13_pred'] = top13_pred_out
            df1['top15_pred'] = top15_pred_out
            df1.to_csv(file_to_save, index=False)

            LR_sch.step()

        top_1[0,idx] = running_top1_acc[-1]
        top_3[0,idx] = running_top3_acc[-1]
        top_5[0,idx] = running_top5_acc[-1]
        top_7[0,idx] = running_top7_acc[-1]
        top_9[0,idx] = running_top9_acc[-1]
        top_11[0,idx] = running_top11_acc[-1]
        top_13[0,idx] = running_top13_acc[-1]
        top_15[0,idx] = running_top15_acc[-1]

```````````````````````````````````````````````````````
Training size is 1
Epoch No. 1
Training-Batch No.100
Loss = 4.49623966217041
Training-Batch No.200
Loss = 3.918278217315674
Training-Batch No.300
Loss = 4.962121486663818
Training-Batch No.400
Loss = 3.248656749725342
Training-Batch No.500
Loss = 4.278253555297852
Training-Batch No.600
Loss = 3.7459192276000977
Training-Batch No.700
Loss = 2.783893346786499
Training-Batch No.800
Loss = 4.446086883544922
Training-Batch No.900
Loss = 3.133831262588501
Training-Batch No.1000
Loss = 3.8383102416992188
Training-Batch No.1100
Loss = 2.6668524742126465
Training-Batch No.1200
Loss = 3.154496192932129
Training-Batch No.1300
Loss = 2.3489136695861816
Training-Batch No.1400
Loss = 4.294672966003418
Training-Batch No.1500
Loss = 4.383076190948486
Training-Batch No.1600
Loss = 3.645030975341797
Training-Batch No.1700
Loss = 3.820612907409668
Training-Batch No.1800
Loss = 4.410951614379883
Training-Batch No.1900
Loss = 3.248305559158325
Trainin

In [22]:
print('Start Validation')
ave_top1_acc = 0
ave_top3_acc = 0
ave_top5_acc = 0
ave_top7_acc = 0
ave_top9_acc = 0
ave_top11_acc = 0
ave_top13_acc = 0
ave_top15_acc = 0
ind_ten = t.as_tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14], device='cuda:0')
top1_pred_out = []
top3_pred_out = []
top5_pred_out = []
top7_pred_out = []
top9_pred_out = []
top11_pred_out = []
top13_pred_out = []
top15_pred_out = []
running_top1_acc = []
running_top3_acc = []
running_top5_acc = []
running_top7_acc = []
running_top9_acc = []
running_top11_acc = []
running_top13_acc = []
running_top15_acc = []
total_count = 0

gt_beam = []

for val_count, (pos_data, beam_val) in enumerate(val_loader):
    net.eval()
    data = [pos_data[0], pos_data[1].type(torch.Tensor), pos_data[2].type(torch.Tensor)]
    label = beam_val.type(torch.LongTensor)
    x = [data[0].cuda(), data[1].cuda(), data[2].cuda()]
    #print("x size", x.size())
    opt.zero_grad()
    labels = label.cuda()

    gt_beam.append(labels.detach().cpu().numpy()[0].tolist())
    total_count += labels.size(0)
    out = net.forward(x)
    _, top_1_pred = t.max(out, dim=1)
    top1_pred_out.append(top_1_pred.detach().cpu().numpy()[0].tolist())
    sorted_out = t.argsort(out, dim=1, descending=True)

    top_3_pred = t.index_select(sorted_out, dim=1, index=ind_ten[0:3])
    top3_pred_out.append(top_3_pred.detach().cpu().numpy()[0].tolist())

    top_5_pred = t.index_select(sorted_out, dim=1, index=ind_ten[0:5])
    top5_pred_out.append(top_5_pred.detach().cpu().numpy()[0].tolist())

    top_7_pred = t.index_select(sorted_out, dim=1, index=ind_ten[0:7])
    top7_pred_out.append(top_7_pred.detach().cpu().numpy()[0].tolist())

    top_9_pred = t.index_select(sorted_out, dim=1, index=ind_ten[0:9])
    top9_pred_out.append(top_9_pred.detach().cpu().numpy()[0].tolist())

    top_11_pred = t.index_select(sorted_out, dim=1, index=ind_ten[0:11])
    top11_pred_out.append(top_11_pred.detach().cpu().numpy()[0].tolist())

    top_13_pred = t.index_select(sorted_out, dim=1, index=ind_ten[0:13])
    top13_pred_out.append(top_13_pred.detach().cpu().numpy()[0].tolist())

    top_15_pred = t.index_select(sorted_out, dim=1, index=ind_ten[0:15])
    top15_pred_out.append(top_15_pred.detach().cpu().numpy()[0].tolist())

    reshaped_labels = labels.reshape((labels.shape[0], 1))
    tiled_3_labels = reshaped_labels.repeat(1, 3)
    tiled_5_labels = reshaped_labels.repeat(1, 5)
    tiled_7_labels = reshaped_labels.repeat(1, 7)
    tiled_9_labels = reshaped_labels.repeat(1, 9)
    tiled_11_labels = reshaped_labels.repeat(1, 11)
    tiled_13_labels = reshaped_labels.repeat(1, 13)
    tiled_15_labels = reshaped_labels.repeat(1, 15)

    batch_top1_acc = t.sum(top_1_pred == labels, dtype=t.float32)
    batch_top3_acc = t.sum(top_3_pred == tiled_3_labels, dtype=t.float32)
    batch_top5_acc = t.sum(top_5_pred == tiled_5_labels, dtype=t.float32)
    batch_top7_acc = t.sum(top_7_pred == tiled_7_labels, dtype=t.float32)
    batch_top9_acc = t.sum(top_9_pred == tiled_9_labels, dtype=t.float32)
    batch_top11_acc = t.sum(top_11_pred == tiled_11_labels, dtype=t.float32)
    batch_top13_acc = t.sum(top_13_pred == tiled_13_labels, dtype=t.float32)
    batch_top15_acc = t.sum(top_15_pred == tiled_15_labels, dtype=t.float32)

    ave_top1_acc += batch_top1_acc.item()
    ave_top3_acc += batch_top3_acc.item()
    ave_top5_acc += batch_top5_acc.item()
    ave_top7_acc += batch_top7_acc.item()
    ave_top9_acc += batch_top9_acc.item()
    ave_top11_acc += batch_top11_acc.item()
    ave_top13_acc += batch_top13_acc.item()
    ave_top15_acc += batch_top15_acc.item()

print("total examples are", total_count)
running_top1_acc.append(ave_top1_acc / total_count)  # (batch_size * (count_2 + 1)) )
running_top3_acc.append(ave_top3_acc / total_count)
running_top5_acc.append(ave_top5_acc / total_count)
running_top7_acc.append(ave_top7_acc / total_count)
running_top9_acc.append(ave_top9_acc / total_count)
running_top11_acc.append(ave_top11_acc / total_count)
running_top13_acc.append(ave_top13_acc / total_count)
running_top15_acc.append(ave_top15_acc / total_count)

print('Training_size {}--No. of skipped batchess {}'.format(n,skipped_batches))
print('Average Top-1 accuracy {}'.format( running_top1_acc[-1]))
print('Average Top-3 accuracy {}'.format( running_top3_acc[-1]))
print('Average Top-5 accuracy {}'.format( running_top5_acc[-1]))
print('Average Top-7 accuracy {}'.format( running_top7_acc[-1]))
print('Average Top-9 accuracy {}'.format( running_top9_acc[-1]))
print('Average Top-11 accuracy {}'.format( running_top11_acc[-1]))
print('Average Top-13 accuracy {}'.format( running_top13_acc[-1]))
print('Average Top-15 accuracy {}'.format( running_top15_acc[-1]))

print("Saving the predicted value in a csv file")
file_to_save = f'{save_directory}//best_epoch_eval.csv'

indx = np.arange(1, len(top1_pred_out)+1, 1)
df2 = pd.DataFrame()
df2['index'] = indx
df2['link_status'] = gt_beam  # Add the link_status column

df2['top1_pred'] = top1_pred_out
df2['top3_pred'] = top3_pred_out
df2['top5_pred'] = top5_pred_out
df2['top7_pred'] = top7_pred_out
df2['top9_pred'] = top9_pred_out
df2['top11_pred'] = top11_pred_out
df2['top13_pred'] = top13_pred_out
df2['top15_pred'] = top15_pred_out
df2.to_csv(file_to_save, index=False)

Start Validation
total examples are 7440
Training_size 1--No. of skipped batchess 0
Average Top-1 accuracy 0.1385752688172043
Average Top-3 accuracy 0.2318548387096774
Average Top-5 accuracy 0.2884408602150538
Average Top-7 accuracy 0.3407258064516129
Average Top-9 accuracy 0.3959677419354839
Average Top-11 accuracy 0.43185483870967745
Average Top-13 accuracy 0.4685483870967742
Average Top-15 accuracy 0.5005376344086021
Saving the predicted value in a csv file


In [23]:
# Testing

In [24]:
# Load the model checkpoint
image_test_dir = './scenario36_64_img_beam_test.csv'
pos_test_dir = './scenario36_64_pos_beam_test.csv'
lidar_test_dir = './scenario36_64_lidar_beam_test.csv'

# Load the test data
test_data = pd.read_csv(image_test_dir)

# Extract the 'unit1_pwr1_best-beam' data and convert it to a list
link_status_data = test_data['original_unit1_pwr3_best-beam'].tolist()
org = test_data['original_index'].tolist()
pwr_60ghz = test_data['original_unit1_pwr3'].tolist()

checkpoint_path = f'{checkpoint_directory}/fused_transfomer_beam_pred'
model.load_state_dict(torch.load(checkpoint_path))
model.eval()
net = model.cuda()

In [25]:
test_loader = DataLoader(DataFeed(image_test_dir, pos_test_dir,lidar_test_dir, image_transform=image_proc_pipe),
                            batch_size=val_batch_size,
                            #num_workers=8,
                            shuffle=False)

In [26]:
print('Start Testing')
ave_top1_acc = 0
ave_top3_acc = 0
ave_top5_acc = 0
ave_top7_acc = 0
ave_top9_acc = 0
ave_top11_acc = 0
ave_top13_acc = 0
ave_top15_acc = 0
ind_ten = t.as_tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14], device='cuda:0')
top1_pred_out = []
top3_pred_out = []
top5_pred_out = []
top7_pred_out = []
top9_pred_out = []
top11_pred_out = []
top13_pred_out = []
top15_pred_out = []
running_top1_acc = []
running_top3_acc = []
running_top5_acc = []
running_top7_acc = []
running_top9_acc = []
running_top11_acc = []
running_top13_acc = []
running_top15_acc = []
total_count = 0

gt_beam = []

for val_count, (pos_data, beam_val) in enumerate(test_loader):
    net.eval()
    data = [pos_data[0], pos_data[1].type(torch.Tensor),pos_data[2].type(torch.Tensor)]
    label = beam_val.type(torch.LongTensor)
    x = [data[0].cuda(), data[1].cuda(), data[2].cuda()]
    #print("x size", x.size())
    opt.zero_grad()
    labels = label.cuda()

    gt_beam.append(labels.detach().cpu().numpy()[0].tolist())
    total_count += labels.size(0)
    out = net.forward(x)
    _, top_1_pred = t.max(out, dim=1)
    top1_pred_out.append(top_1_pred.detach().cpu().numpy()[0].tolist())
    sorted_out = t.argsort(out, dim=1, descending=True)

    top_3_pred = t.index_select(sorted_out, dim=1, index=ind_ten[0:3])
    top3_pred_out.append(top_3_pred.detach().cpu().numpy()[0].tolist())

    top_5_pred = t.index_select(sorted_out, dim=1, index=ind_ten[0:5])
    top5_pred_out.append(top_5_pred.detach().cpu().numpy()[0].tolist())

    top_7_pred = t.index_select(sorted_out, dim=1, index=ind_ten[0:7])
    top7_pred_out.append(top_7_pred.detach().cpu().numpy()[0].tolist())

    top_9_pred = t.index_select(sorted_out, dim=1, index=ind_ten[0:9])
    top9_pred_out.append(top_9_pred.detach().cpu().numpy()[0].tolist())

    top_11_pred = t.index_select(sorted_out, dim=1, index=ind_ten[0:11])
    top11_pred_out.append(top_11_pred.detach().cpu().numpy()[0].tolist())

    top_13_pred = t.index_select(sorted_out, dim=1, index=ind_ten[0:13])
    top13_pred_out.append(top_13_pred.detach().cpu().numpy()[0].tolist())

    top_15_pred = t.index_select(sorted_out, dim=1, index=ind_ten[0:15])
    top15_pred_out.append(top_15_pred.detach().cpu().numpy()[0].tolist())

    reshaped_labels = labels.reshape((labels.shape[0], 1))
    tiled_3_labels = reshaped_labels.repeat(1, 3)
    tiled_5_labels = reshaped_labels.repeat(1, 5)
    tiled_7_labels = reshaped_labels.repeat(1, 7)
    tiled_9_labels = reshaped_labels.repeat(1, 9)
    tiled_11_labels = reshaped_labels.repeat(1, 11)
    tiled_13_labels = reshaped_labels.repeat(1, 13)
    tiled_15_labels = reshaped_labels.repeat(1, 15)

    batch_top1_acc = t.sum(top_1_pred == labels, dtype=t.float32)
    batch_top3_acc = t.sum(top_3_pred == tiled_3_labels, dtype=t.float32)
    batch_top5_acc = t.sum(top_5_pred == tiled_5_labels, dtype=t.float32)
    batch_top7_acc = t.sum(top_7_pred == tiled_7_labels, dtype=t.float32)
    batch_top9_acc = t.sum(top_9_pred == tiled_9_labels, dtype=t.float32)
    batch_top11_acc = t.sum(top_11_pred == tiled_11_labels, dtype=t.float32)
    batch_top13_acc = t.sum(top_13_pred == tiled_13_labels, dtype=t.float32)
    batch_top15_acc = t.sum(top_15_pred == tiled_15_labels, dtype=t.float32)

    ave_top1_acc += batch_top1_acc.item()
    ave_top3_acc += batch_top3_acc.item()
    ave_top5_acc += batch_top5_acc.item()
    ave_top7_acc += batch_top7_acc.item()
    ave_top9_acc += batch_top9_acc.item()
    ave_top11_acc += batch_top11_acc.item()
    ave_top13_acc += batch_top13_acc.item()
    ave_top15_acc += batch_top15_acc.item()

print("total test examples are", total_count)
running_top1_acc.append(ave_top1_acc / total_count)  # (batch_size * (count_2 + 1)) )
running_top3_acc.append(ave_top3_acc / total_count)
running_top5_acc.append(ave_top5_acc / total_count)
running_top7_acc.append(ave_top7_acc / total_count)
running_top9_acc.append(ave_top9_acc / total_count)
running_top11_acc.append(ave_top11_acc / total_count)
running_top13_acc.append(ave_top13_acc / total_count)
running_top15_acc.append(ave_top15_acc / total_count)

print('Training_size {}--No. of skipped batchess {}'.format(n,skipped_batches))
print('Average Top-1 accuracy {}'.format( running_top1_acc[-1]))
print('Average Top-3 accuracy {}'.format( running_top3_acc[-1]))
print('Average Top-5 accuracy {}'.format( running_top5_acc[-1]))
print('Average Top-7 accuracy {}'.format( running_top7_acc[-1]))
print('Average Top-9 accuracy {}'.format( running_top9_acc[-1]))
print('Average Top-11 accuracy {}'.format( running_top11_acc[-1]))
print('Average Top-13 accuracy {}'.format( running_top13_acc[-1]))
print('Average Top-15 accuracy {}'.format( running_top15_acc[-1]))

print("Saving the predicted value in a csv file")
file_to_save = f'{save_directory}//best_epoch_eval_Test.csv'

# Extract the 'unit1_pwr1_best-beam' data and convert it to a list
# link_status_data = test_data['original_unit1_pwr3_best-beam'].tolist()
# org = test_data['original_index'].tolist()
# pwr_60ghz = test_data['original_unit1_pwr3'].tolist()

indx = test_data.index + 1
df2 = pd.DataFrame()
df2['index'] = org
df2['link_status'] = link_status_data  # Add the link_status column
df2['original_unit1_pwr3'] = pwr_60ghz # Add the original_unit1_pwr_60ghz column

df2['top1_pred'] = top1_pred_out
df2['top3_pred'] = top3_pred_out
df2['top5_pred'] = top5_pred_out
df2['top7_pred'] = top7_pred_out
df2['top9_pred'] = top9_pred_out
df2['top11_pred'] = top11_pred_out
df2['top13_pred'] = top13_pred_out

df2['top15_pred'] = top15_pred_out
df2.to_csv(file_to_save, index=False)

Start Testing
total test examples are 2480
Training_size 1--No. of skipped batchess 0
Average Top-1 accuracy 0.14637096774193548
Average Top-3 accuracy 0.23911290322580644
Average Top-5 accuracy 0.29274193548387095
Average Top-7 accuracy 0.34596774193548385
Average Top-9 accuracy 0.39838709677419354
Average Top-11 accuracy 0.43709677419354837
Average Top-13 accuracy 0.4717741935483871
Average Top-15 accuracy 0.4963709677419355
Saving the predicted value in a csv file
